In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from matplotlib import pyplot as plt

import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [11]:
type(trainset.targets)
type(trainset.data)

numpy.ndarray

In [12]:
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=True)
# testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

fg1,fg2,fg3 = 0,1,2

In [13]:
is_fg = [np.where(np.array(trainset.targets)==fg1)[0] , np.where(np.array(trainset.targets)==fg2)[0], np.where(np.array(trainset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   29    30    35 ... 49987 49991 49995] (15000,)


In [14]:
trainset.data = trainset.data[is_fg]
trainset.data.shape

(15000, 32, 32, 3)

In [16]:
trainset.targets = np.array(trainset.targets)[is_fg]
trainset.targets.shape

(15000,)

In [17]:
is_fg = [np.where(np.array(testset.targets)==fg1)[0] , np.where(np.array(testset.targets)==fg2)[0], np.where(np.array(testset.targets)==fg3)[0]  ]
# print(is_fg)
is_fg = np.concatenate(is_fg,axis=0)
print(is_fg, (is_fg).shape)

[   3   10   21 ... 9970 9982 9989] (3000,)


In [18]:
testset.data = testset.data[is_fg]
testset.data.shape

(3000, 32, 32, 3)

In [19]:
testset.targets = np.array(testset.targets)[is_fg]
testset.targets.shape

(3000,)

In [23]:
np.unique(np.array(testset.targets)) , np.unique(np.array(trainset.targets))

(array([0, 1, 2]), array([0, 1, 2]))

In [25]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=False)

In [28]:
import torch.nn as nn
import torch.nn.functional as F

class Classify(nn.Module):
  def __init__(self):
    super(Classify, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 12, 5)
    self.conv3 = nn.Conv2d(12,20,5)
    self.fc1 = nn.Linear(20 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 80)
    self.fc3 = nn.Linear(80, 40)
    self.fc4 = nn.Linear(40,10)
    self.fc5 = nn.Linear(10,3)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = (F.relu(self.conv2(x)))
    x = (F.relu(self.conv3(x)))
    x = x.view(-1, 20 * 6 * 6)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = self.fc5(x)
    return x

In [31]:
what_net = Classify()#.double()
what_net = what_net.to("cuda")

In [32]:
what_net

Classify(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(12, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=720, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=80, bias=True)
  (fc3): Linear(in_features=80, out_features=40, bias=True)
  (fc4): Linear(in_features=40, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=3, bias=True)
)

In [34]:
import torch.optim as optim
criterion_what = nn.CrossEntropyLoss()
optimizer_what = optim.SGD(what_net.parameters(), lr=0.01, momentum=0.9)

In [36]:
acti = []
loss_curi = []
epochs = 1000
for epoch in range(epochs): # loop over the dataset multiple times
    ep_lossi = []

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to("cuda"),labels.to("cuda")

        # zero the parameter gradients
        optimizer_what.zero_grad()

        # forward + backward + optimize
        outputs = what_net(inputs)
        loss = criterion_what(outputs, labels)
        loss.backward()
        optimizer_what.step()

        # print statistics
        running_loss += loss.item()
        mini_batch = 50
        if i % mini_batch == mini_batch-1:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / mini_batch))
            ep_lossi.append(running_loss/mini_batch) # loss per minibatch
            running_loss = 0.0
            
    if(np.mean(ep_lossi) <= 0.001):
      break;
    loss_curi.append(np.mean(ep_lossi))   #loss per epoch            

print('Finished Training')

[1,    50] loss: 1.102
[2,    50] loss: 1.099
[3,    50] loss: 1.099
[4,    50] loss: 1.099
[5,    50] loss: 1.098
[6,    50] loss: 1.098
[7,    50] loss: 1.098
[8,    50] loss: 1.096
[9,    50] loss: 1.091
[10,    50] loss: 1.075
[11,    50] loss: 1.011
[12,    50] loss: 0.918
[13,    50] loss: 0.852
[14,    50] loss: 0.769
[15,    50] loss: 0.641
[16,    50] loss: 0.561
[17,    50] loss: 0.525
[18,    50] loss: 0.489
[19,    50] loss: 0.493
[20,    50] loss: 0.423
[21,    50] loss: 0.430
[22,    50] loss: 0.409
[23,    50] loss: 0.385
[24,    50] loss: 0.367
[25,    50] loss: 0.368
[26,    50] loss: 0.339
[27,    50] loss: 0.329
[28,    50] loss: 0.309
[29,    50] loss: 0.310
[30,    50] loss: 0.297
[31,    50] loss: 0.278
[32,    50] loss: 0.284
[33,    50] loss: 0.265
[34,    50] loss: 0.260
[35,    50] loss: 0.249
[36,    50] loss: 0.240
[37,    50] loss: 0.237
[38,    50] loss: 0.229
[39,    50] loss: 0.209
[40,    50] loss: 0.208
[41,    50] loss: 0.189
[42,    50] loss: 0.177
[

In [38]:
torch.save(what_net.state_dict(),"/content/drive/My Drive/Research/Cheating_data/Classify_net_weights/classify_net"+"cnn"+".pt")

In [39]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels = images.to("cuda"), labels.to("cuda")
        outputs = what_net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d train images: %d %%' % (total,  100 * correct / total))
print(total,correct)

Accuracy of the network on the 15000 train images: 100 %
15000 15000


In [40]:
correct = 0
total = 0
out = []
pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to("cuda"),labels.to("cuda")
        out.append(labels.cpu().numpy())
        outputs= what_net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total))
print(total,correct)

Accuracy of the network on the 10000 test images: 87 %
3000 2623
